In [4]:
!pip install opencv-python
!pip install ultralytics


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: C:\Python312\python.exe -m pip install --upgrade pip


  Using cached ultralytics-8.3.78-py3-none-any.whl.metadata (35 kB)
  Using cached numpy-2.1.1-cp312-cp312-win_amd64.whl.metadata (59 kB)
  Using cached matplotlib-3.10.0-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
  Using cached scipy-1.15.2-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached torch-2.6.0-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached torchvision-0.21.0-cp312-cp312-win_amd64.whl.metadata (6.3 kB)
  Using cached psutil-7.0.0-cp37-abi3-win_amd64.whl.metadata (23 kB)
  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
  Using cached contourpy-1.3.1-cp312-cp312-win_amd64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.56.0-cp312-cp312-win_amd64.whl.metadata (103 kB)
 

ERROR: Could not install packages due to an OSError: [WinError 2] O sistema não pode encontrar o arquivo especificado: 'C:\\Python312\\Scripts\\f2py.exe' -> 'C:\\Python312\\Scripts\\f2py.exe.deleteme'


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: C:\Python312\python.exe -m pip install --upgrade pip


In [5]:
import os
import cv2
from ultralytics import YOLO
import time

# Carrega o modelo pré-treinado
model = YOLO("best.pt")

In [6]:
class_counter = [0, 0, 0, 0]
classes = ['deitado', 'sentado', 'em pé', 'não detectado']
counter=0
amostragem=0.1

cap = cv2.VideoCapture(0)                           # Inicia a webcam
while cap.isOpened():
    ret, frame = cap.read()                         # Captura o frame
    capture_time = time.time()
    if not ret:
        break
    results = model(frame, conf=0.5, verbose=False) # Realiza a inferência no frame capturado
    annotated_frame = results[0].plot()             # Obtém a imagem com as bounding boxes desenhadas
    cv2.imshow("YOLO Detection", annotated_frame)   # Exibe a imagem

    if len(results[0].boxes) > 0:                   # Adiciona tempo na de maior confiança, se abaixo da conf adiciona em não detectado
        best_box = results[0].boxes[results[0].boxes.conf.argmax()]
        class_id = int(best_box.cls)
        confidence = float(best_box.conf)
        bbox = best_box.xyxy[0].tolist()
        #print(f"Classe mais confiante: {results[0].names[class_id]}, Confiança: {confidence:.2f}, Bounding Box: {bbox}")
        class_counter[class_id]+=amostragem+time.time()-capture_time
    else:
        class_counter[3]+=amostragem+time.time()-capture_time
    counter+=0.1+time.time()-capture_time
    time.sleep(amostragem)                          # Tempo entre as capturas
    if cv2.waitKey(1) & 0xFF == ord('q'):           # Pressionar 'q' para sair
        break
cap.release()                                       # Libera a webcam e fecha as janelas
cv2.destroyAllWindows()

# Formatação de Log
for i in range(0, 4):
    print(f'Tempo {classes[i]}: {(class_counter[i]/60):.2f} minutos ({(100 * class_counter[i] / counter):.2f}% do tempo total)')
print(f'Duração total: {(counter/60):.2f} minutos')

Tempo deitado: 0.00 minutos (0.00% do tempo total)
Tempo sentado: 0.00 minutos (0.00% do tempo total)
Tempo em pé: 0.05 minutos (35.97% do tempo total)
Tempo não detectado: 0.09 minutos (64.03% do tempo total)
Duração total: 0.13 minutos
